# ACM-ICPC 2019 アジア地区予選横浜 問題B: Estimating the flood risk

公式の過去問はここに公開されている：https://icpc.iisf.or.jp/2019-yokohama/wp-content/uploads/sites/6/2019/11/all.pdf

概要をざっくり説明すると，

- 正方形の土地が，東西方向に$W$個，南北方向に$D$個ならんでいて，
- そのうちのいくつかの土地の高さはわかっていて，
- どの土地も東西南北の隣との高さの差は最大で1であるとき，
- セルの高さの合計の，可能な最小値を求めよ，

という問題である．

## 線形最適化ソルバーを用いた解法

ここでは，まず，与えられた問題を線形最適化問題として定式化し，線形最適化ソルバーで解いてみる．

セルの東西方向の個数を$W$，セルの南北方向の個数を$D$とする．
高さがわかっているセルの集合を$L$とし，そのそれぞれの座標$x, y$と高さ$Z$がわかっているとする．

このとき，（高さがわかっている土地も含めて）それぞれの土地の高さを変数$z_{x,y}$とすると，与えられた問題は以下の線形最適化問題として表せる．

\begin{align}
\text{min.} \quad & \sum_{x=1}^W \sum_{y=1}^D z_{x,y} \\
\text{s. t.} \quad & z_{x,y} - z_{x + 1, y} \le 1 & (x = 1, \dots, W - 1, \ y = 1, \dots, D), \\
& z_{x,y} - z_{x + 1, y} \ge -1 & (x = 1, \dots, W - 1, \ y = 1, \dots, D), \\
& z_{x,y} - z_{x, y + 1} \le 1 & (x = 1, \dots, W, \ y = 1, \dots, D - 1), \\
& z_{x,y} - z_{x, y + 1} \ge -1 & (x = 1, \dots, W, \ y = 1, \dots, D - 1), \\
& z_{x,y} =  Z_{x, y} & (x = 1, \dots, W, \ y = 1, \dots, D - 1), \\
& z_{x, y} \in \mathbb{Q} & (x = 1, \dots, W, \ y = 1, \dots, D).
\end{align}

ここで，$\mathbb{Q}$は有理数の集合である．
この定式化を制約だけから素直に解釈すると，土地の高さとして整数でない値が出てしまうかもしれないと思える．
しかし，隣接する土地の高さは「差が1まで許される」ことと目的関数を合わせて考えると，最適解には整数しか使われないことになる．

念の為に，
\begin{align}
& | x - y | \le 1 \\
& \Longleftrightarrow -1 \le x - y, \ x - y \le 1
\end{align}
であることに注意されたい．

この定式化をもとに，PuLPを使って解を出力する関数を以下に定義する．

In [2]:
import pulp

def solution(w, d, land):
    lp = pulp.LpProblem() # 線形最適化問題のインスタンスの生成，名前は省略，デフォルトで最小化
    var = {}
    for x in range(1, w + 1):
        for y in range(1, d + 1):
            var[(x, y)] = pulp.LpVariable(f'z{x},{y}') # すべての土地に対応する変数を用意
    lp += pulp.LpAffineExpression([(var[(x, y)], 1) for x in range(1, w + 1) for y in range(1, d + 1)])
    # このLpAffineExpressionは引数に[(x1, a1), (x2, a2), ..., (xn, an)]というリストを与えると
    # a1 x1 + a2 x2 + ... + an xnという1次式を作ってくれる
    for x in range(1, w + 1):
        for y in range(1, d + 1):
            if x < w:
                lp += var[(x, y)] - var[(x + 1, y)] <= 1
                lp += var[(x, y)] - var[(x + 1, y)] >= -1
            if y < d:
                lp += var[(x, y)] - var[(x, y + 1)] <= 1
                lp += var[(x, y)] - var[(x, y + 1)] >= -1
    for x, y, z in land:
        lp += var[(x, y)] == z
    status = lp.solve()
    if status == -1:
        return 'No'
    return round(sum([var[xy].value() for xy in var.keys()]))

In [1]:
# pip install pulp

     |████████████████████████████████| 40.6 MB 14.0 MB/s 
     |████████████████████████████████| 575 kB 8.5 MB/s 
Note: you may need to restart the kernel to use updated packages.


In [3]:
solution(5, 4, [[1, 1, 10], [5, 4, 3]])

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/j0/tkdlg4n54w74ht8tklyh79640000gn/T/c45ff9623e80473da7bcbfbc110df1cf-pulp.mps branch printingOptions all solution /var/folders/j0/tkdlg4n54w74ht8tklyh79640000gn/T/c45ff9623e80473da7bcbfbc110df1cf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 69 COLUMNS
At line 216 RHS
At line 281 BOUNDS
At line 302 ENDATA
Problem MODEL has 64 rows, 20 columns and 126 elements
Coin0008I MODEL read with 0 errors
Presolve 48 (-16) rows, 17 (-3) columns and 96 (-30) elements
Perturbing problem by 0.001 % of 2 - largest nonzero change 0.0009576079 (% 0.09576079) - largest zero change 0
0  Obj 128.44153 Primal inf 0.39999606 (4)
15  Obj 130.04214
Optimal - objective value 130
After Postsolve, objective 130, infeasibilities - dual 0 (0), primal 0 (0)
Optimal 

130

In [4]:
# 入力データのファイルを読み込んで，出力をファイルに書き出す関数を定義する
def answer(input_file_name, output_file_name):
    with open(input_file_name) as input_file, open(output_file_name, 'w') as output_file:
        w, d, n = map(int, input_file.readline().split())
        land = []
        for i in range(n):
            land.append(list(map(int, input_file.readline().split())))
        output_file.write(f'{solution(w, d, land)}\n')
    return

## 線形最適化ソルバーを使わない解法

ちなみにプログラミングコンテストでは，線形最適化ソルバーの利用は想定されていない．

想定解は貪欲解法だと思われる